In [15]:
import  pystan
import argparse
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from tqdm import tqdm
from codebase.data import get_normal_data
from codebase.ibis import (
    compile_model,
    run_mcmc,
    sample_prior_particles,
    essl,
    resample_particles
)
from codebase.ibis_tlk2 import (
    get_weights,
    jitter,
)
from scipy.special import logsumexp
import pdb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
class Arguments:
    gen_model = 0
    existing_directory = 'log/20201108_181656_m3/'
    task_handle = 'm3'
    model_num = 3

args = Arguments()


In [17]:

############################################################
###### Create Directory or Open existing ##########
if args.existing_directory is None:
    log_dir = make_folder(args.task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = args.existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)

nsim_data = 40
data = get_normal_data(nsim_data, 6, 1)
save_obj(data, 'data', log_dir)

param_names = ['alpha', 'Marg_cov', 'L_R']
nsim_particles = 1000


if args.gen_model:
    compile_model(
        model_num=args.model_num,
        prior = False,
        log_dir = log_dir
        )


particles = sample_prior_particles(
    data = data,
    gen_model = args.gen_model,
    model_num = args.model_num,
    param_names = param_names,
    num_samples = nsim_particles, 
    num_chains = 1, 
    log_dir = log_dir
    )


particles['M'] = nsim_particles
particles['param_names'] = ['alpha', 'Marg_cov', 'L_R']
particles['latent_var_names'] = []
particles['names'] = particles['latent_var_names'] + particles['param_names']
particles['w'] = np.zeros(nsim_particles)
log_weights = particles['w']

log_lklhds = np.empty(nsim_data)
degeneracy_limit = 0.5
for t in tqdm(range(nsim_data)):
    log_incr_weights = get_weights(data['y'][t], particles)
    log_lklhds[t] =  logsumexp(log_incr_weights + log_weights) - logsumexp(log_weights)
    log_weights = log_incr_weights + log_weights
    
    if (essl(log_weights) < degeneracy_limit * particles['M']) and (t+1) < data['N']:
        data_temp = dict()
        data_temp['J'] = data['J']
        data_temp['y'] = data['y'][:(t+1)].copy()
        data_temp['N'] = t+1
        print("Deg %d"%(t))
        particles = resample_particles(particles)
        
        particles = jitter(data_temp, particles, log_dir)
        particles['w'] = np.zeros(particles['M'])
    else:
        particles['w'] = log_weights
    save_obj(particles, 'particles%s'%(t+1), log_dir)
    
save_obj(log_lklhds, 'log_lklhds', log_dir)

marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.2f'%marg_lklhd)


  0%|          | 0/40 [00:00<?, ?it/s]



Reading from existing directory: log/20201108_181656_m3/
Deg 0


  2%|▎         | 1/40 [00:58<37:47, 58.13s/it]

Deg 1


  5%|▌         | 2/40 [01:58<37:09, 58.67s/it]

...

Deg 37


 95%|█████████▌| 38/40 [41:29<02:21, 70.78s/it]

Deg 38


100%|██████████| 40/40 [42:32<00:00, 63.81s/it]

Marginal Likelihood 0.00


In [18]:
print('Marginal Likelihood %.5f'%marg_lklhd)


Marginal Likelihood 0.00313
